In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder 
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import joblib # Para guardar preprocesadores
import os
import json # Para guardar nombres de features

In [ ]:
# --- Rutas y Nombres de Archivo (Configurables) ---
RUTA_DATOS_ENRIQUECIDOS_TRAIN_INPUT = './datos/output/tiendas_train_completo_con_imu.gpkg'
LAYER_TRAIN_INPUT = 'tiendas_completo_con_imu'

RUTA_DATOS_ENRIQUECIDOS_TEST_OFICIAL_INPUT = './datos/output/tiendas_TEST_OFICIAL_enriquecido.gpkg'
LAYER_TEST_OFICIAL_INPUT = 'tiendas_test_oficial_enriquecido' # Ajusta si el nombre de la capa es diferente

RUTA_DATOS_PROCESADOS_OUTPUT = './datos/output/procesado_para_modelado/'
if not os.path.exists(RUTA_DATOS_PROCESADOS_OUTPUT):
    os.makedirs(RUTA_DATOS_PROCESADOS_OUTPUT)
    print(f"Directorio creado: {RUTA_DATOS_PROCESADOS_OUTPUT}")
# --- Fin de Configuración ---

print("--- Iniciando Preprocesamiento Final para Modelado ---")

In [ ]:
# --- 1. Cargar Datasets Enriquecidos ---
try:
    df_train_gdf = gpd.read_file(RUTA_DATOS_ENRIQUECIDOS_TRAIN_INPUT, layer=LAYER_TRAIN_INPUT)
    print(f"Datos de entrenamiento enriquecidos cargados: {len(df_train_gdf)} tiendas.")
    
    df_test_oficial_gdf = gpd.read_file(RUTA_DATOS_ENRIQUECIDOS_TEST_OFICIAL_INPUT, layer=LAYER_TEST_OFICIAL_INPUT)
    print(f"Datos de TEST OFICIAL enriquecidos cargados: {len(df_test_oficial_gdf)} tiendas.")
except Exception as e:
    print(f"Error al cargar los archivos GeoPackage enriquecidos: {e}")
    raise

In [ ]:
# --- 2. Selección Inicial de Características y Separación Target/Features ---
cols_a_eliminar = [
    'TIENDA_ID', 
    'geometry', 
    'VENTA_PROMEDIO_MENSUAL', # Solo en train_gdf
    'Meta_venta',             # Solo en train_gdf
    'N_MESES_CON_VENTA_EN_PERIODO', # Solo en train_gdf
    'CVEGEO', 
    'NOM_ENT', 'NOM_MUN', 'NOM_LOC', 'NOM_AGEB', # Nombres geográficos
    'CVE_AGEB ENT', # Si se coló del IMU y es redundante
    
    # NUEVAS COLUMNAS A AÑADIR PARA ELIMINAR (basado en tu warning)
    'AGEB',         # Del warning
    'DATASET',      # Del warning, definitivamente no es una feature
    'ENTIDAD',      # Del warning
    'LOC',          # Del warning
    'MUN',          # Del warning
    'PROMEDIO TOTAL', # Del warning
    'TOTAL'           # Del warning
    # ... y cualquier otra columna identificadora o de texto que no sea feature ...
]

In [ ]:
# Usar df_train_gdf como ejemplo de DataFrame
df = df_train_gdf

# Mostrar columnas con más del 50% de valores nulos
null_threshold = 0.5
null_cols = df.columns[df.isnull().mean() > null_threshold].tolist()
print("Columnas con más del 50% de valores nulos:", null_cols)

# Columnas con un solo valor único
constant_cols = [col for col in df.columns if df[col].nunique() <= 1]
print("Columnas constantes (solo un valor):", constant_cols)

# Columnas dominadas por un solo valor
low_variability_cols = [
    col for col in df.columns
    if df[col].value_counts(normalize=True, dropna=False).values[0] > 0.95
]
print("Columnas con más del 95% del mismo valor:", low_variability_cols)

# Columnas comúnmente irrelevantes para un modelo
likely_useless = ['TIENDA_ID', 'LATITUD_NUM', 'LONGITUD_NUM', 'geometry']
print("Columnas marcadas como no útiles:", likely_useless)

# Eliminar columnas del DataFrame
cols_to_drop = cols_a_eliminar  # Usa la lista ya definida
df_clean = df.drop(columns=cols_to_drop, errors='ignore')

# Verifica el nuevo tamaño
print("Nuevas dimensiones del dataset limpio:", df_clean.shape)


In [ ]:
# --- Procesamiento para el conjunto de ENTRENAMIENTO ---
print("\n--- Procesando datos de ENTRENAMIENTO ---")
cols_a_eliminar_train_existentes = [col for col in cols_a_eliminar if col in df_train_gdf.columns]
df_train_pd = pd.DataFrame(df_train_gdf.drop(columns=cols_a_eliminar_train_existentes))

if 'EXITO' not in df_train_pd.columns:
    raise ValueError("La columna objetivo 'EXITO' no se encuentra en el DataFrame de entrenamiento.")
y_train_full = df_train_pd['EXITO']
X_train_full_raw = df_train_pd.drop(columns=['EXITO'])
print(f"Features de entrenamiento (X_train_full_raw) shape: {X_train_full_raw.shape}")

# --- Procesamiento para el conjunto de TEST OFICIAL ---
print("\n--- Procesando datos de TEST OFICIAL ---")
cols_a_eliminar_test_existentes = [col for col in cols_a_eliminar if col in df_test_oficial_gdf.columns]
# Asegurarse de no intentar eliminar 'EXITO' si no existe en el test set
cols_a_eliminar_test_final = [c for c in cols_a_eliminar_test_existentes if c in df_test_oficial_gdf.columns and c != 'EXITO']

X_test_oficial_raw = pd.DataFrame(df_test_oficial_gdf.drop(columns=cols_a_eliminar_test_final))
print(f"Features de TEST OFICIAL (X_test_oficial_raw) shape: {X_test_oficial_raw.shape}")

# Asegurar consistencia de columnas entre X_train_full_raw y X_test_oficial_raw
# (X_test_oficial_raw no tendrá 'EXITO', lo cual es correcto)
common_features = X_train_full_raw.columns.intersection(X_test_oficial_raw.columns)
missing_in_test = X_train_full_raw.columns.difference(X_test_oficial_raw.columns)
missing_in_train = X_test_oficial_raw.columns.difference(X_train_full_raw.columns)

if len(missing_in_test) > 0:
    print(f"ADVERTENCIA: Columnas en TRAIN que NO están en TEST OFICIAL: {missing_in_test.tolist()}")
    # Esto podría ser un problema. Revisa el enriquecimiento del test.
    # Por ahora, alinearemos X_test_oficial_raw a las columnas de X_train_full_raw, imputando faltantes si es necesario.
    for col in missing_in_test: X_test_oficial_raw[col] = np.nan 
if len(missing_in_train) > 0:
    print(f"ADVERTENCIA: Columnas en TEST OFICIAL que NO están en TRAIN: {missing_in_train.tolist()}")
    X_test_oficial_raw = X_test_oficial_raw.drop(columns=missing_in_train) # Eliminar extras del test

X_test_oficial_raw = X_test_oficial_raw[X_train_full_raw.columns] # Asegurar mismo orden y columnas
print(f"Shapes después de alinear columnas: X_train_full_raw: {X_train_full_raw.shape}, X_test_oficial_raw: {X_test_oficial_raw.shape}")

In [ ]:
# Asegurar consistencia de columnas entre X_train_full_raw y X_test_oficial_raw
# (X_test_oficial_raw no tendrá 'EXITO', lo cual es correcto)
common_features = X_train_full_raw.columns.intersection(X_test_oficial_raw.columns)
missing_in_test = X_train_full_raw.columns.difference(X_test_oficial_raw.columns)
missing_in_train = X_test_oficial_raw.columns.difference(X_train_full_raw.columns)

if len(missing_in_test) > 0:
    print(f"ADVERTENCIA: Columnas en TRAIN que NO están en TEST OFICIAL: {missing_in_test.tolist()}")
    # Esto podría ser un problema. Revisa el enriquecimiento del test.
    # Por ahora, alinearemos X_test_oficial_raw a las columnas de X_train_full_raw, imputando faltantes si es necesario.
    for col in missing_in_test: X_test_oficial_raw[col] = np.nan 
if len(missing_in_train) > 0:
    print(f"ADVERTENCIA: Columnas en TEST OFICIAL que NO están en TRAIN: {missing_in_train.tolist()}")
    X_test_oficial_raw = X_test_oficial_raw.drop(columns=missing_in_train) # Eliminar extras del test

X_test_oficial_raw = X_test_oficial_raw[X_train_full_raw.columns] # Asegurar mismo orden y columnas
print(f"Shapes después de alinear columnas: X_train_full_raw: {X_train_full_raw.shape}, X_test_oficial_raw: {X_test_oficial_raw.shape}")


In [ ]:
print("\n--- Limpieza y Conversión de Tipos de Datos para Features ---")

# Lista de columnas que SABEMOS que deberían ser numéricas pero podrían tener strings
# Esta lista debe ser exhaustiva, basada en tu conocimiento de los datos del Censo e IMU
# ¡¡¡REVISA Y COMPLETA ESTA LISTA CON TODAS LAS COLUMNAS NUMÉRICAS DEL CENSO/IMU!!!
columnas_a_convertir_a_numerico = [
    'POBTOT', 'POBFEM', 'POBMAS', 'P_0A2', 'P_0A2_F', 'P_0A2_M', 'P_3YMAS', 'P_3YMAS_F', 'P_3YMAS_M',
    'P_5YMAS', 'P_5YMAS_F', 'P_5YMAS_M', 'P_12YMAS', 'P_12YMAS_F', 'P_12YMAS_M', 'P_15YMAS', 'P_15YMAS_F',
    'P_15YMAS_M', 'P_18YMAS', 'P_18YMAS_F', 'P_18YMAS_M', 'P_3A5', 'P_3A5_F', 'P_3A5_M', 'P_6A11',
    'P_6A11_F', 'P_6A11_M', 'P_8A14', 'P_8A14_F', 'P_8A14_M', 'P_12A14', 'P_12A14_F', 'P_12A14_M',
    'P_15A17', 'P_15A17_F', 'P_15A17_M', 'P_18A24', 'P_18A24_F', 'P_18A24_M', 'P_15A49_F', 'P_60YMAS',
    'P_60YMAS_F', 'P_60YMAS_M', 'REL_H_M', 'POB0_14', 'POB15_64', 'POB65_MAS', 'PROM_HNV', 'PNACENT',
    'PNACENT_F', 'PNACENT_M', 'PNACOE', 'PNACOE_F', 'PNACOE_M', 'PRES2015', 'PRES2015_F', 'PRES2015_M',
    'PRESOE15', 'PRESOE15_F', 'PRESOE15_M', 'P3YM_HLI', 'P3YM_HLI_F', 'P3YM_HLI_M', 'P3HLINHE',
    'P3HLINHE_F', 'P3HLINHE_M', 'P3HLI_HE', 'P3HLI_HE_F', 'P3HLI_HE_M', 'P5_HLI', 'P5_HLI_NHE',
    'P5_HLI_HE', 'PHOG_IND', 'POB_AFRO', 'POB_AFRO_F', 'POB_AFRO_M', 'PCON_DISC', 'PCDISC_MOT',
    'PCDISC_VIS', 'PCDISC_LENG', 'PCDISC_AUD', 'PCDISC_MOT2', 'PCDISC_MEN', 'PCON_LIMI', 'PCLIM_CSB',
    'PCLIM_VIS', 'PCLIM_HACO', 'PCLIM_OAUD', 'PCLIM_MOT2', 'PCLIM_RE_CO', 'PCLIM_PMEN', 'PSIND_LIM',
    'P3A5_NOA', 'P3A5_NOA_F', 'P3A5_NOA_M', 'P6A11_NOA', 'P6A11_NOAF', 'P6A11_NOAM', 'P12A14NOA',
    'P12A14NOAF', 'P12A14NOAM', 'P15A17A', 'P15A17A_F', 'P15A17A_M', 'P18A24A', 'P18A24A_F',
    'P18A24A_M', 'P8A14AN', 'P8A14AN_F', 'P8A14AN_M', 'P15YM_AN', 'P15YM_AN_F', 'P15YM_AN_M',
    'P15YM_SE', 'P15YM_SE_F', 'P15YM_SE_M', 'P15PRI_IN', 'P15PRI_INF', 'P15PRI_INM', 'P15PRI_CO',
    'P15PRI_COF', 'P15PRI_COM', 'P15SEC_IN', 'P15SEC_INF', 'P15SEC_INM', 'P15SEC_CO', 'P15SEC_COF',
    'P15SEC_COM', 'P18YM_PB', 'P18YM_PB_F', 'P18YM_PB_M', 'GRAPROES', 'GRAPROES_F', 'GRAPROES_M',
    'PEA', 'PEA_F', 'PEA_M', 'PE_INAC', 'PE_INAC_F', 'PE_INAC_M', 'POCUPADA', 'POCUPADA_F',
    'POCUPADA_M', 'PDESOCUP', 'PDESOCUP_F', 'PDESOCUP_M', 'PSINDER', 'PDER_SS', 'PDER_IMSS',
    'PDER_ISTE', 'PDER_ISTEE', 'PAFIL_PDOM', 'PDER_SEGP', 'PDER_IMSSB', 'PAFIL_IPRIV', 'PAFIL_OTRAI',
    'P12YM_SOLT', 'P12YM_CASA', 'P12YM_SEPA', 'PCATOLICA', 'PRO_CRIEVA', 'POTRAS_REL', 'PSIN_RELIG',
    'TOTHOG', 'HOGJEF_F', 'HOGJEF_M', 'POBHOG', 'PHOGJEF_F', 'PHOGJEF_M', 'VIVTOT', 'TVIVHAB',
    'TVIVPAR', 'VIVPAR_HAB', 'VIVPARH_CV', 'TVIVPARHAB', 'VIVPAR_DES', 'VIVPAR_UT', 'OCUPVIVPAR',
    'PROM_OCUP', 'PRO_OCUP_C', 'VPH_PISODT', 'VPH_PISOTI', 'VPH_1DOR', 'VPH_2YMASD', 'VPH_1CUART',
    'VPH_2CUART', 'VPH_3YMASC', 'VPH_C_ELEC', 'VPH_S_ELEC', 'VPH_AGUADV', 'VPH_AEASP', 'VPH_AGUAFV',
    'VPH_TINACO', 'VPH_CISTER', 'VPH_EXCSA', 'VPH_LETR', 'VPH_DRENAJ', 'VPH_NODREN', 'VPH_C_SERV',
    'VPH_NDEAED', 'VPH_DSADMA', 'VPH_NDACMM', 'VPH_SNBIEN', 'VPH_REFRI', 'VPH_LAVAD', 'VPH_HMICRO',
    'VPH_AUTOM', 'VPH_MOTO', 'VPH_BICI', 'VPH_RADIO', 'VPH_TV', 'VPH_PC', 'VPH_TELEF', 'VPH_CEL',
    'VPH_INTER', 'VPH_STVP', 'VPH_SPMVPI', 'VPH_CVJ', 'VPH_SINRTV', 'VPH_SINLTC', 'VPH_SINCINT',
    'VPH_SINTIC',
    'IM_2020', 'IMN_2020' # GM_2020 es categórica
    # No olvides las columnas de conteo y distancia de DENUE y OSM, esas ya deberían ser numéricas.
    # Ej: 'denue_conteo_competidores_directos_conv_200m', 'denue_dist_competidores_directos_conv_cercano_m', etc.
]

# Aplicar a X_train_full_raw
for col in columnas_a_convertir_a_numerico:
    if col in X_train_full_raw.columns:
        # Reemplazar no numéricos (como '*') con NaN para que to_numeric funcione
        X_train_full_raw[col] = X_train_full_raw[col].replace('*', np.nan) 
        X_train_full_raw[col] = pd.to_numeric(X_train_full_raw[col], errors='coerce') # coerce pone NaN si no puede convertir
    else:
        print(f"Advertencia: Columna '{col}' para convertir a numérico no encontrada en X_train_full_raw.")

# Aplicar a X_test_oficial_raw
for col in columnas_a_convertir_a_numerico:
    if col in X_test_oficial_raw.columns:
        X_test_oficial_raw[col] = X_test_oficial_raw[col].replace('*', np.nan)
        X_test_oficial_raw[col] = pd.to_numeric(X_test_oficial_raw[col], errors='coerce')
    else:
        print(f"Advertencia: Columna '{col}' para convertir a numérico no encontrada en X_test_oficial_raw.")

print("Conversión a numérico completada (con 'coerce' para errores).")
print("Tipos de datos en X_train_full_raw después de conversión:")
print(X_train_full_raw[columnas_a_convertir_a_numerico].info())

In [ ]:
# --- 3. Identificar Tipos de Columnas (Numéricas y Categóricas) ---
# Primero identifica todas las que Pandas considera numéricas
todas_columnas_numericas_inicial = X_train_full_raw.select_dtypes(include=np.number).columns.tolist()
todas_columnas_categoricas_inicial = X_train_full_raw.select_dtypes(include=['object', 'category']).columns.tolist()

# --- AJUSTE AQUÍ PARA PLAZA_CVE ---
# Definir las columnas que SON verdaderamente categóricas
columnas_categoricas = ['NIVELSOCIOECONOMICO_DES', 'ENTORNO_DES', 
                        'SEGMENTO_MAESTRO_DESC', 'LID_UBICACION_TIENDA', 
                        'GM_2020', 
                        'PLAZA_CVE'] # Añadimos PLAZA_CVE aquí

# Las numéricas serán todas las demás que Pandas leyó como número, EXCEPTO PLAZA_CVE
columnas_numericas = [col for col in todas_columnas_numericas_inicial if col not in ['PLAZA_CVE']]
# También asegúrate de que ninguna de las otras categóricas definidas arriba esté en la lista de numéricas por error.
columnas_numericas = [col for col in columnas_numericas if col not in columnas_categoricas]


# Antes de pasar PLAZA_CVE al OneHotEncoder, si es numérico (1, 2, 3...), 
# es buena idea convertirlo a string/object para que OHE lo trate correctamente como categorías distintas.
# Esto se hace sobre X_train_full_raw y X_test_oficial_raw
if 'PLAZA_CVE' in X_train_full_raw.columns:
    X_train_full_raw['PLAZA_CVE'] = X_train_full_raw['PLAZA_CVE'].astype(str)
if 'PLAZA_CVE' in X_test_oficial_raw.columns:
    X_test_oficial_raw['PLAZA_CVE'] = X_test_oficial_raw['PLAZA_CVE'].astype(str)
# --- FIN AJUSTE ---


print(f"\nColumnas Numéricas DEFINITIVAS ({len(columnas_numericas)}): {columnas_numericas}")
print(f"Columnas Categóricas DEFINITIVAS ({len(columnas_categoricas)}): {columnas_categoricas}")

# Imprimir cardinalidad de las categóricas DEFINITIVAS
print("\n--- Cardinalidad de Columnas Categóricas DEFINITIVAS (en X_train_full_raw) ---")
for col in columnas_categoricas:
    if col in X_train_full_raw.columns:
        num_unique = X_train_full_raw[col].nunique(dropna=False)
        print(f"Columna Categórica: '{col}', Número de Valores Únicos: {num_unique}")
    else:
        print(f"Advertencia: La columna categórica '{col}' no se encontró en X_train_full_raw.")


In [ ]:
# --- 4. Crear Pipelines de Preprocesamiento ---
pipeline_numerico = Pipeline([
    ('imputer_num', SimpleImputer(strategy='median')), # Imputar con mediana para robustez a outliers
    ('scaler', StandardScaler())
])

pipeline_categorico = Pipeline([
    # Paso 1: Imputar NaNs con una constante (string)
    ('imputer_nan_to_str', SimpleImputer(strategy='constant', fill_value='Desconocido')), 
    # Paso 2: El OneHotEncoder se encargará de esta nueva categoría "Desconocido"
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocesador = ColumnTransformer([
    ('num', pipeline_numerico, columnas_numericas),
    ('cat', pipeline_categorico, columnas_categoricas)
], remainder='drop') # 'drop' para eliminar columnas no especificadas (si las hubiera)
                    # o 'passthrough' si quieres mantenerlas tal cual (menos común)
print("\nPipeline de preprocesamiento creado.")

In [ ]:
# --- 5. Dividir DATOS DE ENTRENAMIENTO en Entrenamiento y Validación Interna ---
X_train_raw_split, X_val_raw_split, y_train, y_val = train_test_split(
    X_train_full_raw, y_train_full, 
    test_size=0.2, 
    random_state=42, 
    stratify=y_train_full
)
print(f"\nDatos de entrenamiento divididos: X_train_raw_split: {X_train_raw_split.shape}, X_val_raw_split: {X_val_raw_split.shape}")

In [ ]:
print("\n--- Análisis de Cardinalidad de Columnas Categóricas (en X_train_raw_split) ---")
for col in columnas_categoricas:
    if col in X_train_raw_split.columns:
        num_unique = X_train_raw_split[col].nunique()
        print(f"Columna: '{col}', Número de Valores Únicos: {num_unique}")
        if num_unique > 50: # Un umbral arbitrario para señalar alta cardinalidad
            print(f"  ¡ALERTA! Alta cardinalidad en '{col}'. Valores de ejemplo: {X_train_raw_split[col].unique()[:5]}")
    else:
        print(f"Columna '{col}' no encontrada en X_train_raw_split para análisis de cardinalidad.")

In [ ]:
# --- 6. Ajustar el Preprocesador SOLO con X_train_raw_split ---
print("\nAjustando el preprocesador con X_train_raw_split...")
preprocesador.fit(X_train_raw_split)
print("Preprocesador ajustado.")


In [ ]:
# --- 7. Aplicar el Preprocesador a Todos los Conjuntos de Features ---
print("\nTransformando X_train_raw_split...")
X_train_procesado_arr = preprocesador.transform(X_train_raw_split)
print("Transformando X_val_raw_split...")
X_val_procesado_arr = preprocesador.transform(X_val_raw_split)
print("Transformando X_test_oficial_raw...")
X_test_oficial_procesado_arr = preprocesador.transform(X_test_oficial_raw)

In [ ]:
# --- 8. Obtener Nombres de Features y Convertir a DataFrames (Opcional pero Recomendado) ---
try:
    nombres_features_onehot = preprocesador.named_transformers_['cat']['onehot']\
                               .get_feature_names_out(columnas_categoricas)
    columnas_finales_procesadas = columnas_numericas + list(nombres_features_onehot)
    
    X_train_df = pd.DataFrame(X_train_procesado_arr, columns=columnas_finales_procesadas, index=X_train_raw_split.index)
    X_val_df = pd.DataFrame(X_val_procesado_arr, columns=columnas_finales_procesadas, index=X_val_raw_split.index)
    X_test_oficial_df = pd.DataFrame(X_test_oficial_procesado_arr, columns=columnas_finales_procesadas, index=X_test_oficial_raw.index)

    print(f"\nShapes de DataFrames procesados: X_train_df: {X_train_df.shape}, X_val_df: {X_val_df.shape}, X_test_oficial_df: {X_test_oficial_df.shape}")
    print("Primeras filas de X_train_df procesado:")
    print(X_train_df.head())
except Exception as e:
    print(f"Error al obtener nombres de features o crear DataFrames procesados: {e}")
    print("Los datos X_train, X_val, X_test_oficial seguirán como arrays de NumPy.")
    # Si falla, los guardaremos como .npy y guardaremos los nombres de columna por separado si es posible
    X_train_df, X_val_df, X_test_oficial_df = X_train_procesado_arr, X_val_procesado_arr, X_test_oficial_procesado_arr
    if 'nombres_features_onehot' in locals(): # Intenta guardar nombres de features de todas formas
         columnas_finales_procesadas = columnas_numericas + list(nombres_features_onehot)

In [ ]:
# --- 9. Guardar los Conjuntos de Datos Procesados y el Preprocesador ---
print("\n--- Guardando Datos Procesados y Preprocesador ---")
try:
    if isinstance(X_train_df, pd.DataFrame):
        X_train_df.to_csv(RUTA_DATOS_PROCESADOS_OUTPUT + 'X_train_procesado.csv', index=False)
        X_val_df.to_csv(RUTA_DATOS_PROCESADOS_OUTPUT + 'X_val_procesado.csv', index=False)
        X_test_oficial_df.to_csv(RUTA_DATOS_PROCESADOS_OUTPUT + 'X_test_oficial_procesado.csv', index=False)
    else: # Guardar como .npy si son arrays
        np.save(RUTA_DATOS_PROCESADOS_OUTPUT + 'X_train_procesado.npy', X_train_df)
        np.save(RUTA_DATOS_PROCESADOS_OUTPUT + 'X_val_procesado.npy', X_val_df)
        np.save(RUTA_DATOS_PROCESADOS_OUTPUT + 'X_test_oficial_procesado.npy', X_test_oficial_df)

    y_train.to_csv(RUTA_DATOS_PROCESADOS_OUTPUT + 'y_train.csv', index=False, header=True)
    y_val.to_csv(RUTA_DATOS_PROCESADOS_OUTPUT + 'y_val.csv', index=False, header=True)
    
    # Guardar los nombres de las columnas procesadas (muy útil para los notebooks de modelado)
    if 'columnas_finales_procesadas' in locals():
        with open(RUTA_DATOS_PROCESADOS_OUTPUT + 'columnas_procesadas_finales.json', 'w') as f:
            json.dump(columnas_finales_procesadas, f)
        print("Nombres de columnas procesadas guardados en 'columnas_procesadas_finales.json'")
    
    joblib.dump(preprocesador, RUTA_DATOS_PROCESADOS_OUTPUT + 'preprocesador_completo.joblib')
    print(f"Preprocesador guardado en '{RUTA_DATOS_PROCESADOS_OUTPUT}preprocesador_completo.joblib'")
    print("\nConjuntos de datos listos para modelado guardados.")

except Exception as e:
    print(f"Error al guardar los archivos procesados o el preprocesador: {e}")

print("\n--- Preprocesamiento Final Completado ---")
print(f"Los artefactos para modelado están en: {RUTA_DATOS_PROCESADOS_OUTPUT}")